<a href="https://colab.research.google.com/github/pagnihot/pa-pub-test/blob/main/Portfolio_Risk_MC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import auth
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

In [2]:
if GOOGLE_COLAB:
    !pip install google-cloud-pubsub

In [3]:
if GOOGLE_COLAB:
    auth.authenticate_user()

In [4]:
from google import auth
from google.cloud import storage
from google.cloud import pubsub_v1

import datetime, time
import subprocess
import argparse
import os, sys
import json

In [5]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import datetime , time
from math import sqrt
from scipy.stats import norm
import yfinance as yf

In [53]:
STOCK="AMZN"
short_window = 40
long_window = 100
trading_days = 500
sim_num = 100000
portfolio_stocks_list = ["GOOG","MSFT","NFLX","TSLA","IBM","AMZN"]
portfolio_capital=100000
lookback_days=730

In [7]:
import uuid
sim_id = str(uuid.uuid4())

In [8]:
period_end=datetime.datetime.now()

In [9]:
period_start = period_end - datetime.timedelta(days=lookback_days)

In [10]:
yf.pdr_override()
stock_df = pdr.get_data_yahoo(STOCK,start=period_start, end=period_end)

[*********************100%%**********************]  1 of 1 completed


In [11]:
# Calculate the compound annual growth rate (CAGR) which
# will give us our mean return input (mu)
days = (stock_df.index[-1] - stock_df.index[0]).days
cagr = ((((stock_df['Adj Close'][-1]) / stock_df['Adj Close'][1])) ** (365.0/days)) - 1
mu = cagr

In [12]:
stock_df['Returns'] = stock_df['Adj Close'].pct_change()
vol = stock_df['Returns'].std()*sqrt(252)

In [13]:
initial_capital= float(portfolio_capital)

In [57]:
# Set up empty list to hold our ending values for each simulated price series
sim_result = []

# Set up empty list to hold portfolio value for each simulated price serries, this is the value of position['total']
portfolio_total = []

In [15]:
# Define Variables
start_price = stock_df['Adj Close'][-1] #starting stock price (i.e. last available real stock price)

In [24]:
from typing import Callable

# Define GCP Environment
project_id = "fsif-330919"
topic_id = "QU1"

publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)
publish_futures = []

In [17]:
def get_callback(publish_future: pubsub_v1.publisher.futures.Future, data: str) -> Callable[[pubsub_v1.publisher.futures.Future], None]:

  def pub_callback(publish_future: pubsub_v1.publisher.futures.Future) -> None:
    try:
      # Wait 60 seconds for the publish call to succeed.
      print(publish_future.result(timeout=60))
    except futures.TimeoutError:
      print(f"Publishing {data} timed out.")

  return pub_callback

In [18]:
def sub_callback(message: pubsub_v1.subscriber.message.Message) -> None:
  """Process received message"""
  print(f"Received message: ID={message.message_id} Data={message.data}")
  print(f"[{datetime.datetime.now()}] Processing: {message.message_id}")
  sim_data = json.loads(message.data)
  sim_num          = sim_data["sim_num"]
  sim_result.append(sim_data["stock_price"])
  portfolio_total.append(sim_data["portfolio_value"])
  print(f"[{datetime.datetime.now()}] Processed: {message.message_id}")
  message.ack()

In [54]:
for i in range(sim_num):
  data = f'{{ "sim_id": "{sim_id}", "sim_num": {i}, "stock": "{STOCK}", "short_window": {short_window}, "long_window": {long_window}, "trading_days": {trading_days}, "start_price": {start_price}, "lookback_days": {lookback_days}, "mu": {mu}, "vol": {vol}, "initial_capital": {initial_capital}  }}'
  #print(data)
  publish_future = publisher.publish(topic_path, data.encode("utf-8"))
  # Non-blocking. Publish failures are handled in the callback function.
  publish_future.add_done_callback(get_callback(publish_future, data))
  publish_futures.append(publish_future)

print(f"Published {i} messages with error handler to {topic_path}.")


Streaming output truncated to the last 5000 lines.
{ "sim_id": "b1a172c8-720d-4da2-97ff-186919463432", "sim_num": 97513, "stock": "AMZN", "short_window": 40, "long_window": 100, "trading_days": 500, "start_price": 133.08999633789062, "lookback_days": 730, "mu": -0.10375197793841895, "vol": 0.42759959137708875, "initial_capital": 100000.0  }
{ "sim_id": "b1a172c8-720d-4da2-97ff-186919463432", "sim_num": 97514, "stock": "AMZN", "short_window": 40, "long_window": 100, "trading_days": 500, "start_price": 133.08999633789062, "lookback_days": 730, "mu": -0.10375197793841895, "vol": 0.42759959137708875, "initial_capital": 100000.0  }
{ "sim_id": "b1a172c8-720d-4da2-97ff-186919463432", "sim_num": 97515, "stock": "AMZN", "short_window": 40, "long_window": 100, "trading_days": 500, "start_price": 133.08999633789062, "lookback_days": 730, "mu": -0.10375197793841895, "vol": 0.42759959137708875, "initial_capital": 100000.0  }
{ "sim_id": "b1a172c8-720d-4da2-97ff-186919463432", "sim_num": 97516, "st

In [55]:
from google.api_core import retry

In [56]:
r_topic_id = "QU1R"
subscription_id = f'{r_topic_id}-sub1'

subscriber = pubsub_v1.SubscriberClient()
subscription_path = subscriber.subscription_path(project_id, subscription_id)

In [44]:
"""
streaming_pull_future = subscriber.subscribe(subscription_path, callback=sub_callback,max_messages=sim_num)
print(f"Pulling messages from {subscription_path}..., in project:{project_id}")

with subscriber:
  try:
    streaming_pull_future.result()
  except Exception as e:
    print(e)
"""

'\nstreaming_pull_future = subscriber.subscribe(subscription_path, callback=sub_callback,max_messages=sim_num)\nprint(f"Pulling messages from {subscription_path}..., in project:{project_id}")\n\nwith subscriber:\n  try:\n    streaming_pull_future.result()\n  except Exception as e:\n    print(e)\n'

In [47]:
with subscriber:
  # The subscriber pulls a specific number of messages. The actual
  # number of messages pulled may be smaller than max_messages.
  response = subscriber.pull(
    request={"subscription": subscription_path, "max_messages": sim_num},
    retry=retry.Retry(deadline=300),
  )

  if len(response.received_messages) != 0:
    ack_ids = []
    for received_message in response.received_messages:
      print(f"Received: {received_message.message.data}.")
      sim_data = json.loads(received_message.message.data)
      sim_num  = sim_data["sim_num"]
      sim_result.append(sim_data["stock_price"])
      portfolio_total.append(sim_data["portfolio_value"])

    ack_ids.append(received_message.ack_id)
    # Acknowledges the received messages so they will not be sent again.
    subscriber.acknowledge(request={"subscription": subscription_path, "ack_ids": ack_ids})

    print(f"Received and acknowledged {len(response.received_messages)} messages from {subscription_path}.")
  else:
    print(f"Received {len(response.received_messages)} messages from {subscription_path}.")


Received: b'{ "sim_num": 70, "stock_price": 82.01105434543317, "portfolio_value": 97803.9225662648 }'.
Received: b'{ "sim_num": 81, "stock_price": 72.55058498341361, "portfolio_value": 96995.23538734262 }'.
Received: b'{ "sim_num": 78, "stock_price": 208.802656201046, "portfolio_value": 108887.99609505323 }'.
Received: b'{ "sim_num": 88, "stock_price": 67.07831430794525, "portfolio_value": 99648.7312043566 }'.
Received: b'{ "sim_num": 25, "stock_price": 212.2396700115838, "portfolio_value": 105101.17351228585 }'.
Received: b'{ "sim_num": 75, "stock_price": 188.50901304808826, "portfolio_value": 102664.21182342622 }'.
Received: b'{ "sim_num": 77, "stock_price": 59.74094037042223, "portfolio_value": 100591.22378964633 }'.
Received: b'{ "sim_num": 95, "stock_price": 160.28926942332296, "portfolio_value": 101799.57074330068 }'.
Received: b'{ "sim_num": 89, "stock_price": 69.8616761586023, "portfolio_value": 97267.37184804831 }'.
Received: b'{ "sim_num": 99, "stock_price": 62.71132630515691

In [48]:
#create histogram of ending stock values for our mutliple simulations
plt.rcParams["figure.figsize"] = (9,8)
plt.hist(sim_result,bins=50)
plt.suptitle('Monte Carlo Simulation Results', fontsize=20)
plt.xlabel('Stock Value $')
plt.show()

#create histogram of ending protfolio values for our mutliple simulations, compare this value to "initial_capital"
plt.hist(portfolio_total,bins=50)
plt.suptitle('Portfolio Simulated Results (based on $100K investment / ' + str(trading_days) + ' days)', fontsize=20)
plt.xlabel('Portfolio Value $')
plt.show()

NameError: ignored